In [1]:
from Mediapipe import MediapipeHands
import os

initial_dir = os.getcwd()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [2]:
mediapipeHands = MediapipeHands()

In [3]:
training_dir = f'{initial_dir}/data/asl_letters_lensegua_c'
mediapipeHands.extract_images_coordinates_from_dir(training_dir)

Hay 774 imagenes a extraer coordenadas
>> No se detectaron las dos manos necesarias en  /Users/andreaamaya/Trabajo/megaproyecto_model/data/asl_letters_lensegua_c/ñ (3) 2.png  con id  389
>> No se detectaron las dos manos necesarias en  /Users/andreaamaya/Trabajo/megaproyecto_model/data/asl_letters_lensegua_c/q (1) 2.png  con id  477
>> No se detectaron las dos manos necesarias en  /Users/andreaamaya/Trabajo/megaproyecto_model/data/asl_letters_lensegua_c/q (4) 2.png  con id  491
>> No se detectaron las dos manos necesarias en  /Users/andreaamaya/Trabajo/megaproyecto_model/data/asl_letters_lensegua_c/q (6).png  con id  497
>> No se detectaron las dos manos necesarias en  /Users/andreaamaya/Trabajo/megaproyecto_model/data/asl_letters_lensegua_c/q (7).png  con id  500
>> No se detectaron las dos manos necesarias en  /Users/andreaamaya/Trabajo/megaproyecto_model/data/asl_letters_lensegua_c/x (9) 2.png  con id  686
>> No se detectaron las dos manos necesarias en  /Users/andreaamaya/Trabajo/

In [4]:
validation_dir = f'{initial_dir}/data/letters_val'
mediapipeHands.extract_images_coordinates_from_dir(validation_dir, is_val=True)

Hay 24 imagenes a extraer coordenadas


In [5]:
df = mediapipeHands.get_padded_data()

In [6]:
df.head()

,sequence_id,target,file,x_Left_hand_0,y_Left_hand_0,x_Left_hand_1,y_Left_hand_1,x_Left_hand_2,y_Left_hand_2,x_Left_hand_3,...,x_Right_hand_16,y_Right_hand_16,x_Right_hand_17,y_Right_hand_17,x_Right_hand_18,y_Right_hand_18,x_Right_hand_19,y_Right_hand_19,x_Right_hand_20,y_Right_hand_20
0,1,a,a (1) 2.png,0.000000,0.048423,0.064452,0.326796,0.334537,0.619979,0.491786,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,a,a (1) 3.png,0.000000,0.168434,0.215612,0.370625,0.470939,0.601284,0.569410,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,a,a (1).png,0.423459,0.140339,0.655401,0.363291,0.737979,0.605570,0.726230,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,a,a (10) 2.png,0.204003,0.300461,0.476328,0.473332,0.767077,0.663757,0.826010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,a,a (10) 3.png,0.920469,0.121714,0.639250,0.352007,0.432306,0.607345,0.365829,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
validation_sequence_ids = mediapipeHands.validation_sequence_ids

In [8]:
if (len(validation_sequence_ids) > 0):
    validation_path = f'{initial_dir}/data/validation_letters.csv'
    validation_df = df[df['sequence_id'].isin(validation_sequence_ids)][["sequence_id", "target", "file"]]
    
    train_path = f'{initial_dir}/data/train_letters.csv'
    train_df = df[~df['sequence_id'].isin(validation_sequence_ids)][["sequence_id", "target", "file"]]

    validation_df.to_csv(validation_path, index=False)
    train_df.to_csv(train_path, index=False)

In [9]:
data_path = f'{initial_dir}/data/data_letters.csv'
df_final = df.drop("file", axis=1)

df_final.to_csv(data_path, index=False)

In [10]:
mediapipeHands = MediapipeHands()
extra_val_dir = f'{initial_dir}/data/test hola'
mediapipeHands.extract_images_coordinates_from_dir(extra_val_dir)

df = mediapipeHands.get_padded_data()

Hay 4 imagenes a extraer coordenadas


In [11]:
data_path = f'{initial_dir}/data/data_test_hola.csv'
df.to_csv(data_path, index=False)